In [1]:

from keras.models import Sequential  # Modelo sequencial
from keras.layers import Dense, Flatten, BatchNormalization  
from keras.layers import Conv2D, MaxPooling2D, Dropout  
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold, cross_val_score

import numpy as np
import pandas as pd
import cv2 
import os 
import glob


In [2]:
'''
    Este trecho do código tem a função de para mudar a semente geradora dos
    numeros aleatórios. Será utilizado para obter os mesmos resultados ao
    fazer manualmente a divisão da base de dados durante a operação
    de validação cruzada
'''
seed = 5
np.random.seed(seed)

In [3]:
''' 
    Carregando a pasta de imagens (Dataset dos ovos)
    * A função glob encontra todos os nomes de caminhos que correspondem a um padrão 
      especificado de acordo com as regras usadas pelo shell Unix

    * Para cada um dos arquivos encontrados é feita a leitura
      do arquivo .jpg (imagem), as imagens lidas são armazenadas
      na lista 'data'
'''

img_dir = "C:/Users/marci/Desktop/Projeto mestrado/CNN Egg application/Egg Dataset"
data_path = os.path.join(img_dir,'*g') 

folder = glob.glob(data_path) 
data = [] 
for files in folder: 
    img = cv2.imread(files) 
    data.append(img) 

In [4]:
'''
    Carregar a base de dados
    Obs: para a validação cruzada, apenas os valores de treinamento serão utilizadas,
    ou seja -> (xTrain, yTrain)
'''

xTrain = np.array(data)
yTrain = pd.read_csv("RealData.csv")

print('Shape dos atributos previsores de treinamento', xTrain.shape)
print('Shape das classes de treinamento', yTrain.shape)


Shape dos atributos previsores de treinamento (5100, 512, 512, 3)
Shape das classes de treinamento (5100, 1)


In [5]:
'''
    Alterando o tipo dos dados para float32 a fim de aplicar 
    a normalização (min/max normalization) para de que os valores 
    dos pixels estejam entre 0 e 1, tornando o processamento mais eficiente
    obs: 255 é o valor máximo do pixel
'''

xTrain = xTrain.astype('float32')
xTrain /= 255

In [6]:
def create_model():
    
    '''
    Definição da rede neural convolucional

    * Criação da cnn no modelo sequencial (sequencia de layers)
    * Criação de duas camadas de convolução com função de ativação Relu, 
      seguidas pelos processos de normalização dos mapas de características e max Pooling.
      O processo de flattening é adicionado ao final das camadas. 
    '''

    classifier = Sequential()

    classifier.add(Conv2D(32, (3,3), input_shape = (512, 512, 3), activation = 'relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2,2)))

    classifier.add(Conv2D(32, (3,3), activation = 'relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2,2)))

    classifier.add(Flatten())

    '''
        * Adição das hidden Layers
        * Aplicação da operação de dropout às saídas 
            Busca zerar uma determinada quantidade de entradas 
            a fim de otimizar o sistema e reduzir o overfitting
        * Adição da camada de saída, utilizando a sigmóide como função de ativação
            A sigmóide é utilzada em classificações binárias
            OBS -> units = 1 indica que há apenas uma unidade de saída para
            a classificação binária
    '''
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units = 128, activation = 'relu'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))

    '''
        * Compilação da rede neural
            OBS: Para a classificação binária, a função de perda será definida 
            por 'binary_crossentropy'. Para a classificação em múltiplas classes
            é necessário utilizar o 'categorical_crossentropy'
    '''

    classifier.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return classifier

In [7]:
model = KerasClassifier(build_fn = create_model, epochs=5, batch_size=16, verbose=1)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(model, xTrain, yTrain, cv = kfold)

print("Acurácia média: %.2f%%" % (scores.mean() * 100))
print("Desvio padrão: %.2f%%" % (scores.std() * 100))

c:\Users\marci\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)





Epoch 1/5


287/287 [==============================] - 439s 2s/step - loss: 6.2873 - accuracy: 0.7011
Epoch 2/5
287/287 [==============================] - 425s 1s/step - loss: 0.6128 - accuracy: 0.7309
Epoch 3/5
287/287 [==============================] - 437s 2s/step - loss: 0.5923 - accuracy: 0.7309
Epoch 4/5
287/287 [==============================] - 425s 1s/step - loss: 0.5851 - accuracy: 0.7309
Epoch 5/5
32/32 [==============================] - 11s 312ms/step


c:\Users\marci\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\marci\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\marci\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\marci\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which poi

Acurácia média: nan%
Desvio padrão: nan%


c:\Users\marci\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
9 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\marci\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\marci\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "c:\Users\marci\AppData\Local\Programs\Python\Python311\Li